Segmenting and Clustering Neighborhoods in Toronto

3. To create the above dataframe:

    The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    
    Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    
    More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

    If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
    Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

4. Submit a link to your Notebook on your Github repository. (10 marks)

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml
import geocoder # import geocoder

In [109]:
url = r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [110]:
#pull data from Wikipedia and drop all Borough values that are not assigned

tables = pd.read_html(url, header = 0)
df_all_data = tables[0]
df_fixed_data = df_all_data.drop(df_all_data[df_all_data.Borough == 'Not assigned'].index)
pd.set_option('display.max_colwidth', -1)

In [111]:
#replace unassigned neighborhood values as the Borough value
for row, index in df_fixed_data.iterrows():
    if df_fixed_data.at[row, 'Neighbourhood'] == 'Not assigned':
        df_fixed_data.at[row, 'Neighbourhood'] = index['Borough']
    else:
        continue
    

In [112]:
df_fixed_data

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [124]:
#rename postcode column to align with data downstream, then use the groupby function to group rows based on their postal code and borough values
df_fixed_data = df_fixed_data.rename(columns = {'Postcode':'Postal Code'})
df_fixed_data = df_fixed_data.groupby(['Postal Code', 'Borough'], as_index=False, sort=False).agg(', '.join)


In [125]:
df_fixed_data.shape

(103, 3)

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:

In [116]:
csv = r"C:\Users\JLark\Downloads\Geospatial_Coordinates.csv"

In [126]:
#read the csv into pandas and create a new dataframe object
lat_lon = pd.read_csv(csv)
lat_lon

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [127]:
#merge the two dataframes on the consistent data, the postal code

df_combined = lat_lon.merge(df_fixed_data, how = 'right', on = 'Postal Code')

In [128]:
df_combined


,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
5,M1J,43.744734,-79.239476,Scarborough,Scarborough Village
6,M1K,43.727929,-79.262029,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,43.711112,-79.284577,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,43.716316,-79.239476,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,43.692657,-79.264848,Scarborough,"Birch Cliff, Cliffside West"
